<a href="https://colab.research.google.com/github/LunaEyad/Deep-Learning-Foundations/blob/main/Lectures_7_%26_8_~_Automated_Differentiation_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vanilla python Closed Form Solution


In [ ]:
#generate the points
from random import Random
from math import ceil , sqrt

def generate_pnts(n=1000):
  seed=5
  random_num=Random(x=seed)
  return(
      [random_num.uniform(a=0,b=1)for _ in range(n)],
      [random_num.uniform(a=0,b=1)for _ in range(n)]
  )


#clac grad
def clac_grad(x_p ,y_p , batch_x , batch_y):

  sum_x , sum_y= 0 ,0
  n=len(batch_x)
  for x_i, y_i in zip(batch_x ,batch_y):
    inv_sqrt = ((x_i - x_p) ** 2 + (y_i - y_p) ** 2) ** (-0.5)
    sum_x += inv_sqrt * (x_i - x_p)
    sum_y += inv_sqrt * (y_i - y_p)
  return -sum_x/n, -sum_y/n


#loss fn
def loss_fn(x_p ,y_p , batch_x , batch_y):
 
  return (1/len(batch_x))* sum([sqrt((x_i-x_p)**2+(y_i-y_p)**2) for x_i , y_i in zip(batch_x, batch_y)])

data_x, data_y = generate_pnts(n=1000)
x_p , y_p = 0.3 , 0.3
grad_x , grad_y =clac_grad(x_p, y_p, data_x ,data_y)
curr_loss = loss_fn(x_p, y_p, data_x ,data_y)

print(f"Clsoed form: gradient for x_p = {grad_x}, gradient for y_p = {grad_y}")
print(f"Clsoed form: loss = {curr_loss}")

Clsoed form: gradient for x_p = -0.3277744397151291, gradient for y_p = -0.336282174741702
Clsoed form: loss = 0.4430528244756474


## Pytorch auto-grad engine

In [ ]:
import torch
pnt =torch.tensor([0.3 , 0.3])
pnt.requires_grad = True
pnt.retain_grad()
data_x , data_y =generate_pnts(n=1000)
data=torch.tensor([data_x , data_y])
data=data.t()

for i in range(2):
  loss_torch=torch.mean(torch.sqrt(((data-pnt)**2).sum(dim=1)))
  print(f"torch loss: {loss_torch}")
  loss_torch.backward()
  print(f"torch auto gradient: {pnt.grad.data}")
  pnt.grad.zero_()
  print(f"gradient after zeroing : {pnt.grad.data}")





torch loss: 0.44305282831192017
torch auto gradient: tensor([-0.3278, -0.3363])
gradient after zeroing : tensor([0., 0.])
torch loss: 0.44305282831192017
torch auto gradient: tensor([-0.3278, -0.3363])
gradient after zeroing : tensor([0., 0.])


## Building our own Auto-Grad from scratch

In [ ]:
class comp_node:
  def __init__(self , val , children=[], op="assign"):
    self.val=val
    self.children=children
    self.op=op
    self.grad=0
    self.backward_prop= lambda: None

  def to_comp_node(self, obj):
    if not isinstance(obj, comp_node):
     return comp_node(val = obj)
    else:
      return obj

#node1(self) - 3(other)
  def __sub__(self,other): 
    other=self.to_comp_node(other)
    out= comp_node(self.val-other.val , children=[self , other] , op="sub")
    def __backward_prop():
      self.grad+=out.grad*1
      other.grad+=out.grad*-1

    out.backward_prop=__backward_prop

    return out

#3(other)-node1(self)
  def __rsub__(self,other): 
    other=self.to_comp_node(other)
    return other - self

  def __pow__(self, exponenet):
    if not isinstance (exponenet , (int ,float)):
      raise ValueError ("unsupported type")
    out= comp_node(val=self.val** exponenet , children=[self], op=f"pow of {exponenet}")

    def __backward_prop():
      self.grad+=out.grad* (exponenet*self.val**(exponenet-1))

    out.backward_prop=__backward_prop
    return out
  


  def __add__(self,other): 
    other= self.to_comp_node(other)
    out= comp_node(self.val + other.val , children=[self , other] , op="add")
    def __backward_prop():
      self.grad+=out.grad*1
      other.grad+=out.grad*1

    out.backward_prop=__backward_prop
    return out

  def __radd__(self,other): 
    other=self.to_comp_node(other)
    return other + self

  def __mul__(self,other): 
    other= self.to_comp_node(other)
    out= comp_node(self.val * other.val , children=[self , other] , op="mul")
    def __backward_prop():
       self.grad += out.grad * other.val
       other.grad += out.grad * self.val
    out.backward_prop = __backward_prop 
    return out

  def __rmul__(self,other): 
    other=self.to_comp_node(other)
    return other * self
    
  def __repr__(self): 
    return(f"val={self.val:.4f}, children={len(self.children)}, op={self.op} , grad={self.grad}") 
  


In [ ]:
# Define two comp_node instances
a = comp_node(5)
b = comp_node(10)

# Test subtraction operator
c = a - b
assert c.val == -5

# Test reverse subtraction operator
d = 20 - b
assert d.val == 10

# Test power operator
i = a ** 2
assert i.val == 25


# Test addition operator
c = a + b
assert c.val == 15

# Test reverse addition operator
d = 20 + b
assert d.val == 30


# Test mult operator
c = a * b
assert c.val == 50

# Test reverse mult operator
d = 20 * b
assert d.val == 200

In [ ]:
data_x, data_y = generate_pnts(n=1)
print(data_x, data_y)
x_p , y_p = comp_node(val=0.3) , comp_node(val=0.3)


def loss_graph(x_p, y_p, data_x, data_y):
  I_x , I_y = x_p - data_x , y_p -data_y
  G_x , G_y=  I_x **2 , I_y**2
  M = G_x + G_y
  l=M**0.5
  return  l , [l , M ,G_x , G_y ,I_x , I_y , x_p, y_p]

l , rev_top_sort =loss_graph(x_p, y_p, data_x[0], data_y[0])  
rev_top_sort[0].grad=1

for i , node in enumerate(rev_top_sort):

  node.backward_prop()
  print(i , node)

[0.6229016948897019] [0.7417869892607294]
0 val=0.5472, children=1, op=pow of 0.5 , grad=1
1 val=0.2994, children=2, op=add , grad=0.9137222319490423
2 val=0.1043, children=1, op=pow of 2 , grad=0.9137222319490423
3 val=0.1952, children=1, op=pow of 2 , grad=0.9137222319490423
4 val=-0.3229, children=2, op=sub , grad=-0.5900849147094943
5 val=-0.4418, children=2, op=sub , grad=-0.8073411877467226
6 val=0.3000, children=0, op=assign , grad=-0.5900849147094943
7 val=0.3000, children=0, op=assign , grad=-0.8073411877467226
